In [ ]:
    !pip install albumentations

In [ ]:
from albumentations import Compose,OneOf

---

# <p style="font-family: 'Amiri'; font-size: 3rem; color: Black; text-align: center; margin: 0; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3); background-color: #c9b68b; padding: 20px; border-radius: 20px; border: 7px solid Black; width:95%">4 | Data Augmentation Pipeline For Training </p>

In [ ]:
def Gain(p, y, min_gain_in_db=-15, max_gain_in_db=15):
    
#     对音频信号 y 应用一个随机增益。
    
#     参数:
#     y : np.ndarray
#         音频信号。
#     min_gain_in_db : float
#         增益的最小分贝值。
#     max_gain_in_db : float
#         增益的最大分贝值。
#     p : float
#         应用增益的概率。
        
#     返回:
#     y_gained : np.ndarray
#         应用了增益的音频信号。
   
    if np.random.rand() < p:
        # 随机选择增益值
        gain_db = np.random.uniform(min_gain_in_db, max_gain_in_db)
        # 将分贝转换为线性比例
        gain = 10 ** (gain_db / 20)
        # 应用增益
        y_gained = y * gain
    else:
        y_gained = y
    return y_gained
   
def GainTransition(y, sr, min_gain_in_db=-24.0, max_gain_in_db=6.0, min_duration=0.2, max_duration=6.0, p=1.0):
    
#     在指定的时间段内对音频信号 y 逐渐改变增益。
    
#     参数:
#     y : np.ndarray
#         音频信号。
#     min_gain_in_db : float
#         增益的最小分贝值。
#     max_gain_in_db : float
#         增益的最大分贝值。
#     min_duration : float
#         增益变化的最小持续时间（秒）。
#     max_duration : float
#         增益变化的最大持续时间（秒）。
#     p : float
#         应用增益转换的概率。
#     sr : int
#         采样率。
        
#     返回:
#     y_transitioned : np.ndarray
#         应用了增益转换的音频信号。
    
    if np.random.rand() < p:
        # 计算增益变化的持续时间（样本数）
        duration_samples = int(np.random.uniform(min_duration, max_duration) * sr)
        # 随机选择起始增益和结束增益
        start_gain_db = np.random.uniform(min_gain_in_db, max_gain_in_db)
        end_gain_db = np.random.uniform(min_gain_in_db, max_gain_in_db)
        # 将分贝转换为线性比例
        start_gain = 10 ** (start_gain_db / 20)
        end_gain = 10 ** (end_gain_db / 20)
        
        # 应用增益转换
        t = np.arange(len(y))
        gain = np.interp(t, [0, duration_samples], [start_gain, end_gain])
        y_transitioned = y * gain
    else:
        y_transitioned = y
    return y_transitioned
#增加高斯噪音，输入x为波形wave
def AddGaussianNoise(x,w=0.004,p=1):
    if p > random.random():
        output = x + w * np.random.normal(loc=0, scale=1, size=len(x))
        return output
    else:
        return x
    
#"彩色噪声" 通常指的是具有特定频率分布的噪声   
def AddColorNoise(y, sr, p=1, min_snr_db=5, max_snr_db=20, min_f_decay=0.01, max_f_decay=0.1):
    
#     在音频信号 y 中添加彩色噪声。
    
#     参数:
#     y : np.ndarray
#         音频信号。
#     sr : int
#         采样率。
#     p : float
#         添加噪声的概率。
#     min_snr_db : float
#         最小信噪比，单位为分贝。
#     max_snr_db : float
#         最大信噪比，单位为分贝。
#     min_f_decay : float
#         最小频率衰减。
#     max_f_decay : float
#         最大频率衰减。
        
#     返回:
#     noisy_y : np.ndarray
#         添加了彩色噪声的音频信号。
    if np.random.rand() < p:
        # 随机选择信噪比
        snr_db = np.random.uniform(min_snr_db, max_snr_db)
        
        # 计算信号功率
        signal_power = np.sum(y**2) / len(y)
        
        # 根据信噪比计算噪声功率
        noise_power = signal_power / 10**(snr_db / 10)
        
        # 计算噪声
        noise = np.random.normal(0, np.sqrt(noise_power), y.shape)
        
        # 将噪声添加到信号中
        y_noisy = y + noise
        
        # 频率衰减计算
        f_decay = np.random.uniform(min_f_decay, max_f_decay)
        # 计算频率响应
        freq_response = np.exp(-f_decay * np.arange(0, sr) / sr)
        
        # 应用频率响应到噪声上
        y_noisy = librosa.istft(librosa.stft(y_noisy) * freq_response)
        
    else:
        y_noisy = y

    return y_noisy
    
def PitchShift(x, sr, n_steps, bins_per_octave=12):
    # sr: 音频采样率
    # n_steps: 要移动多少步
    # bins_per_octave: 每个八度音阶(半音)多少步
    if p > random.random():
        return librosa.effects.pitch_shift(x, sr, n_steps, bins_per_octave=bins_per_octave)
    else:
        return x
    
#波形位移,时间单位？
def Shift(x,shift_time,p):
    if p > random.random():
        return np.roll(x, int(shift_time))
    else:
        return x

In [ ]:
if isTrain== True:

    #针对音频增强？
    normal_augment = Compose([
        OneOf([
            Gain(samples, p=1.0, min_gain_in_db=-15, max_gain_in_db=15),
            GainTransition(samples,cfg.sr, min_gain_in_db=-24.0, max_gain_in_db=6.0,
                           min_duration=0.2, max_duration=6.0,  p=1.0)
        ], p=cfg.aug_gain),
        
        OneOf([
            AddGaussianNoise(samples,w=0.004,p=1),
            AddColorNoise(samples, cfg.sr, p=1, min_snr_db=5, max_snr_db=20, min_f_decay=0.01, max_f_decay=0.1)
        ],p=cfg.aug_noise),

    
        PitchShift(samples, cfg.sr, n_steps=6, bins_per_octave=12),
        Shift(samples, shift_time = 2,p=cfg.aug_wave_shift)
    ])
    #针对图像增强？
    alb_transform = [
        albumentations.XYMasking(num_masks_x=2, num_masks_y=1, 
                                 mask_x_length=cfg.size_x//30, mask_y_length=cfg.n_mels//30,
                                 fill_value=0, mask_fill_value=0, p=cfg.aug_spec_xymasking),
        albumentations.CoarseDropout(fill_value=0, min_holes=20, max_holes=50, p=cfg.aug_spec_coarsedrop),
        albumentations.HorizontalFlip(p=cfg.aug_spec_hflip)    
    ]
    albumentations_augment = albumentations.Compose(alb_transform)

---